**Table of contents**<a id='toc0_'></a>    
- [导入模块](#toc1_)    
- [EndNote Term lists](#toc2_)    
- [下载](#toc3_)    
- [保存](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[导入模块](#toc0_)

In [2]:
import pandas as pd 
import logging 

root_logger = logging.getLogger()
# root_logger.setLevel(logging.DEBUG)  # 将root logger级别设置为DEBUG
root_logger.setLevel(logging.WARN)  # 将root logger级别设置为DEBUG
# logger.warning('This is a warning message')  # 输出日志
# logger.debug('This is a debug message')  # 正确输出日志

# <a id='toc2_'></a>[EndNote Term lists](#toc0_)
Term lists是期刊名称全称和简称的对照表： 
* Endnote官方的Medical.txt整理的挺好的，大多数期刊都被收录其中
* 但是仍然不够

In [3]:
term_handle = 'EndNote_Term_Lists\Medical.txt'
term = pd.read_table(term_handle, header=None, names=['Full Name', 'Add 1', 'Add 2', 'Add 3'], index_col=None)
term
# 总共四列

,Full Name,Add 1,Add 2,Add 3
0,New Phytologist,NaN,New Phytol,NaN
1,Nature Communications,NaN,Nat Commun,NaN
2,Nature ecology & evolution,NaN,Nat Ecol Evol,NaN
3,The ISME journal,NaN,ISME J,NaN
4,Environmental microbiome,NaN,Environ Microbiome,NaN
...,...,...,...,...
14508,Zoonoses and Public Health,Zoonoses Public Health,Zoonoses Public Health,Zoonoses & Public Health
14509,Zoonoses Research,Zoonoses Res.,Zoonoses Res,NaN
14510,Zuchthygiene,Zuchthygiene,Zuchthygiene,NaN
14511,ZWR,ZWR,ZWR,NaN


In [4]:
term.shape, term['Full Name'].unique().shape
# 又重复的

((14513, 4), (14501,))

# <a id='toc3_'></a>[下载](#toc0_)
* 从PubMed下载所有Journal的信息
* https://www.ncbi.nlm.nih.gov/nlmcatalog
* https://www.nlm.nih.gov/archive/20130415/tsd/serials/lji.html
* 关注Full name和Addr.

In [5]:
def add2medical(term:pd.DataFrame, ncbi_term_lists)->pd.DataFrame:
    counter = 0
    with open('NCBI_Term_Lists/journalspmc.txt', 'r') as f:
        contents = f.readlines()
        for index, l in enumerate(contents):
            x = l.strip().split(': ')
            if "Title Abbreviation" in x[0]:
                full_name = contents[index+1].strip().split(': ')[1].strip('.')
                logging.info(f"{full_name}\t{x[1]}")            # full name -> add
                logging.info(f"{x[1]}\t{full_name}")            # add -> full name
                if full_name not in term['Full Name'].values:
                    counter += 1
                    logging.info(f"{counter}\t{full_name}\t{x[1]}")
                    print(f"{counter}\t{full_name}\t{x[1]}")
                    app = pd.Series({'Full Name': full_name, 'Add 1': '', 'Add 2': x[1], 'Add 3': ''})
                    # term = term.append(app, ignore_index=True)
                    term = pd.concat([term, app.to_frame().T])
            # break
    return term

In [6]:
new_term = term.copy()
new_term = add2medical(term=new_term, ncbi_term_lists='NCBI_Term_Lists/journalspmc.txt') # add pmc

1	Advances in rehabilitation science and practice	Adv Rehabil Sci Pract
2	Biomolecules & biomedicine	Biomol Biomed
3	Critical care science	Crit Care Sci
4	Discover nano	Discov Nano
5	Global advances in integrative medicine and health	Glob Adv Integr Med Health
6	Journal of conservative dentistry and endodontics	J Conserv Dent Endod
7	Parasites, hosts and diseases	Parasites Hosts Dis
8	Therapeutic advances in allergy and rhinology	Ther Adv Allergy Rhinol
9	Therapeutic advances in pulmonary and critical care medicine	Ther Adv Pulm Crit Care Med
10	Thoracic research and practice	Thorac Res Pract
11	Urology research & practice	Urol Res Pract
12	AJPM focus	AJPM Focus
13	Biotechnology for biofuels and bioproducts	Biotechnol Biofuels Bioprod
14	BJA open	BJA Open
15	BMC primary care	BMC Prim Care
16	BMJ medicine	BMJ Med
17	Cancer innovation	Cancer Innov
18	Cell insight	Cell Insight
19	CJC pediatric and congenital heart disease	CJC Pediatr Congenit Heart Dis
20	Eco-Environment & Health	Eco Envi

In [7]:
new_term.shape

(17987, 4)

In [8]:
new_term = add2medical(term=new_term, ncbi_term_lists='NCBI_Term_Lists\currentlyindexed.txt') # add medical
# 还是四列

In [9]:
new_term.shape

(17987, 4)

# <a id='toc4_'></a>[保存](#toc0_)

In [44]:
new_term.to_csv('New_Term_Lists/New_Medical.txt', sep='\t', index=None, header=None)
# 保存